# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

In [1]:
import pandas as pd
import sqlite3
import os

os.getcwd()

'/home/nick/code/week03/week05'

In [2]:
conn = sqlite3.connect('../../DSI-CHI-1/lessons/week-05/1.3-db-lab/assets/datasets/enron.db')
c = conn.cursor()

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [3]:
results_employee = c.execute('SELECT * FROM EmployeeBase LIMIT 5;')
results_employee.fetchall()

[(1,
  u'John Arnold',
  u'Forestry',
  u'ENA Gas Financial',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (2,
  u'Harry Arora',
  u'Forestry',
  u'ENA East Power',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (3,
  u'Robert Badeer',
  u'Forestry',
  u'ENA West Power',
  u'Mgr Trading',
  u'Male',
  u'Junior'),
 (4,
  u'Susan Bailey',
  u'Legal',
  u'ENA Legal',
  u'Specialist Legal',
  u'Female',
  u'Junior'),
 (5,
  u'Eric Bass',
  u'Forestry',
  u'ENA Gas Texas',
  u'Trader',
  u'Male',
  u'Junior')]

In [4]:
results_message = c.execute('SELECT * FROM MessageBase LIMIT 5;')
results_message.fetchall()

[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138),
 (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138),
 (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138),
 (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138),
 (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)]

In [5]:
results_recipient = c.execute('SELECT * FROM RecipientBase LIMIT 5;')
results_recipient.fetchall()

[(1, 1, 59), (2, 1, 15), (3, 1, 15), (4, 1, 109), (4, 2, 49)]

Import each of the 3 tables to a Pandas Dataframes

In [6]:
employee_df = pd.read_sql('SELECT * FROM EmployeeBase;', con=conn)
employee_df.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [7]:
message_df = pd.read_sql('SELECT * FROM MessageBase;', con=conn)
message_df.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [8]:
recipient_df = pd.read_sql('SELECT * FROM RecipientBase;', con=conn)
recipient_df.head()

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [25]:
import numpy as np
pivot_table = recipient_df.pivot_table(index='mid', values='rno', aggfunc=np.max)
pivot_table.value_counts()

1     14985
2      2962
3      1435
4       873
5       711
6       180
7       176
8        61
13       57
11       47
12       33
10       29
15       28
9        24
14       11
16        9
21        2
17        2
57        2
22        1
52        1
20        1
55        1
19        1
24        1
18        1
49        1
Name: rno, dtype: int64

In [26]:
recipient_df.pivot_table(index='mid', values='to_eid', aggfunc=len).value_counts()

1     14985
2      2962
3      1435
4       873
5       711
6       180
7       176
8        61
13       57
11       47
12       33
10       29
15       28
9        24
14       11
16        9
21        2
17        2
57        2
22        1
52        1
20        1
55        1
19        1
24        1
18        1
49        1
Name: to_eid, dtype: int64

In [9]:
print employee_df.shape, message_df.shape, recipient_df.shape

(156, 7) (21635, 5) (38388, 3)


1. 156 employees in the company 
2. 21635 messages in the database.

In [10]:
message_df.dtypes

mid           int64
filename     object
unix_time     int64
subject      object
from_eid      int64
dtype: object

In [13]:
message_df['datetime'] = pd.to_datetime(message_df['unix_time'], unit='s')
message_df.head()

,mid,filename,unix_time,subject,from_eid,datetime
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138,1998-11-13 04:07:00
1,2,taylor-m/sent/17,911459940,Indemnification,138,1998-11-19 07:19:00
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138,1998-11-19 08:24:00
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138,1998-11-24 02:23:00
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138,1998-11-30 03:22:00


In [14]:
message_df['datetime'].describe()

count                   21635
unique                  21309
top       1999-12-27 03:21:00
freq                        4
first     1998-11-13 04:07:00
last      2002-06-21 13:37:34
Name: datetime, dtype: object

In [15]:
message_df['datetime'].max(), message_df['datetime'].min()

(Timestamp('2002-06-21 13:37:34'), Timestamp('1998-11-13 04:07:00'))

Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.